In [2]:
%matplotlib inline
%timeit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import time
import pickle
import datetime
import re
import pyActigraphy

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import utils as wearutils
from wearables.scripts import data as weardata
from wearables.scripts import train as weartrain
from wearables.scripts import eval_ as weareval
from wearables.scripts import model as wearmodels
from wearables.scripts import DTW as weardtw

import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.spatial.distance import pdist, squareform
import fastdtw
import umap

plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=1
plt.rcParams['savefig.dpi'] = 600
sns.set_style("ticks")

In [3]:
# load data
mfp = '/home/ngrav/project/wearables/model_zoo'
bst_modelpkl = os.path.join(mfp, '213-itv52_InceptionTime_GA5.pkl')
bst_trainer = os.path.join(mfp, 'trainer_itv52_InceptionTime_GA5.pkl')

split = 'train'

X, md_model, trainer = weardtw.data_from_trainer(bst_trainer, split=split)

# call eval to get output, modify for return_hidden = True option
evaluation = weareval.eval_trained(trainer, split=split,
                                   modelpkl=bst_modelpkl,
                                   two_outputs=True)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/torch/nn/modules/conv.py:294: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/aten/src/ATen/native/Convolution.cpp:660.)
  return F.conv1d(input, weight, bias, self.stride,
/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


cohort = pd.read_csv('/home/ngrav/data/wearables/processed/List_1260.csv')
cohort.head()

In [42]:
[i for i in trainer.data.data['IDs'].keys() if len(i.split('_')[0]) != 4]

[]

In [30]:
cohort = pd.read_csv('/home/ngrav/data/wearables/processed/List_1260.csv')


In [32]:
cohort['x'].to_list()

[1001,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1011,
 1012,
 1013,
 1016,
 1017,
 1018,
 1020,
 1021,
 1022,
 1025,
 1028,
 1031,
 1035,
 1038,
 1039,
 1040,
 1042,
 1043,
 1044,
 1046,
 1047,
 1048,
 1049,
 1050,
 1051,
 1052,
 1053,
 1054,
 1056,
 1057,
 1058,
 1059,
 1061,
 1062,
 1065,
 1066,
 1067,
 1068,
 1069,
 1070,
 1071,
 1073,
 1074,
 1076,
 1077,
 1079,
 1080,
 1081,
 1083,
 1085,
 1086,
 1087,
 1088,
 1094,
 1095,
 1096,
 1097,
 1098,
 1100,
 1101,
 1102,
 1103,
 1104,
 1105,
 1106,
 1107,
 1108,
 1110,
 1113,
 1116,
 1117,
 1118,
 1121,
 1122,
 1123,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1133,
 1134,
 1135,
 1136,
 1138,
 1140,
 1143,
 1144,
 1145,
 1146,
 1147,
 1149,
 1150,
 1151,
 1152,
 1154,
 1155,
 1156,
 1157,
 1159,
 1160,
 1163,
 1164,
 1167,
 1168,
 1169,
 1170,
 1171,
 1173,
 1174,
 1176,
 1177,
 1178,
 1180,
 1181,
 1183,
 1184,
 1185,
 1186,
 1187,
 1188,
 1189,
 1190,
 1192,
 1193,
 1194,
 1195,
 1196,
 1197,
 1198,
 1199,
 1200,
 1201,
 1202,
 1203,

In [26]:
dir(trainer.data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'batch_size',
 'data',
 'get_dataloaders',
 'load_preproced',
 'prop_trainset',
 'split_data',
 'target_name',
 'tasktype',
 'test_dl',
 'test_ids',
 'train_dl',
 'train_ids',
 'val_dl',
 'val_ids']

In [ ]:
print('{:d}'

KeyError: 'd'

In [18]:
cohort.dtypes

x    int64
dtype: object

In [12]:
md_model.head()

,record_id,age_enroll,marital,gestage_by,insur,ethnicity,race,bmi_1vis,prior_ptb_all,fullterm_births,...,edinb2_1trim,edinb3_1trim,edinb4_1trim,edinb5_1trim,edinb6_1trim,edinb7_1trim,edinb8_1trim,edinb9_1trim,edinb10_1trim,GA
2376_34,2376,35.0,0.0,2.0,3.0,0.0,1.0,27.772373,0.0,0.0,...,1.0,4.0,3.0,3.0,3.0,3.0,2.0,3.0,1.0,34.0
2589_18,2589,29.0,0.0,2.0,1.0,0.0,1.0,32.236332,1.0,1.0,...,2.0,2.0,3.0,1.0,3.0,3.0,3.0,3.0,1.0,18.0
2418_27,2418,28.0,1.0,2.0,3.0,0.0,0.0,27.405882,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0
1140_36,1140,24.0,0.0,1.0,1.0,1.0,1.0,26.097082,0.0,1.0,...,1.0,3.0,3.0,2.0,1.0,2.0,2.0,2.0,1.0,36.0
2347_35,2347,31.0,0.0,1.0,3.0,1.0,0.0,50.582722,0.0,0.0,...,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,35.0


In [21]:
md_model['record_id'].isin(cohort['x'])

2376_34     True
2589_18     True
2418_27     True
1140_36     True
2347_35     True
           ...  
2322_9      True
2209_15     True
2121_23     True
1227_12    False
1523_36     True
Name: record_id, Length: 1726, dtype: bool

In [22]:
md_model.loc[md_model['record_id'].isin(cohort['x']), : ].shape

(1622, 108)

In [23]:
md_model.shape

(1726, 108)